In [2]:
import torch
from torchvision import datasets, transforms
import pandas as pd
from PIL import Image
import random

In [3]:
class ImageFolderWithCategories(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithCategories, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (int(path.split('\\')[1]),))
        return tuple_with_path
    
class TestLoaderNoClass():
    def __init__(self, url, batch_size):
        self.batch_size = batch_size
        self.test = pd.read_csv(url)

    def getRandomURL(self):
        length = len(self.test)
        rand_num = random.randint(0, length)
        
        path = self.test['Path'][rand_num]
        classid = self.test['ClassId'][rand_num]

        return path, classid

    def Load(self):
        for i in range(self.batch_size):
            path, classid = self.getRandomURL()
            img = Image.open(path)
            print(transforms.ToTensor(img))


testloader = TestLoaderNoClass(url='Test.csv', batch_size=1)
testloader.Load()

In [3]:
# Define a transform to normalize the data

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomCrop(25),
    #We are normalizing the input tensor with a mean of 0.5 and a standard deviation of 0.5. This scales the pixel values to be between -1 and 1, which can help with training stability.
    transforms.Normalize((0.5,), (0.5,))
])

trainset = ImageFolderWithCategories('Train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64)

print(trainloader)

#testset = ImageFolderWithCategories('Test', transform=transform)
#testloader = torch.utils.data.DataLoader(trainset, batch_size=64)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(128 * 3 * 3, 512)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(512, 43)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        
        x = x.view(x.size(0), -1)

        
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.fc2(x)
        
        return x
net = Net(num_classes=42)

In [ ]:
import torch.optim as optim
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

num_epochs = 10

for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, desc=f"Epoch {epoch+1}"), 0):
        
        inputs, labels, category = data

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, category)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 200 == 199:    # print every 200 mini-batches
            print(f"[Epoch {epoch+1}, batch {i+1:5d}] loss: {running_loss/200:.3f}")
            running_loss = 0.0

print('Finished Training')


Epoch 1:  33%|███▎      | 200/613 [00:21<00:35, 11.68it/s]

[Epoch 1, batch   200] loss: 2.691


Epoch 1:  65%|██████▌   | 401/613 [00:40<00:19, 10.73it/s]

[Epoch 1, batch   400] loss: 3.755


Epoch 1:  98%|█████████▊| 601/613 [01:00<00:01, 10.45it/s]

...

[Epoch 10, batch   400] loss: 4.082


Epoch 10:  98%|█████████▊| 602/613 [00:56<00:00, 11.08it/s]

[Epoch 10, batch   600] loss: 3.709


Epoch 10: 100%|██████████| 613/613 [00:57<00:00, 10.58it/s]

Finished Training


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")